# NFL Models

## Data Export
```
DB_FILE=${FANTASY_HOME}/nfl_hist_2009-2022.scored.db
# WR TE
dumpdata.sc --seasons 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 \
    --stats tds "receiving*" --pos WR TE --progress ${DB_FILE} \
    --no_teams --target_calc_stats "*" --hist_recent_games 3 \
    --player_team_stats passing_yds yds pts turnovers \
    --opp_team_stats def_int def_sacks op_passing_yds op_pts op_yds \
    --hist_recent_mode ma --current_extra venue is_home \
    -f nfl_WRTE.csv

# RB
dumpdata.sc --seasons 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 \
    --stats tds "receiving*" "rushing*" --pos RB --progress ${DB_FILE} \
    --no_teams --target_calc_stats "*" --hist_recent_games 3 \
    --player_team_stats rushing_yds yds pts turnovers \
    --opp_team_stats def_int def_sacks op_rushing_yds op_pts op_yds \
    --hist_recent_mode ma --current_extra venue is_home \
    -f nfl_RB.csv

# QB
dumpdata.sc --seasons 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 \
    --stats tds "rushing*" "passing*" --pos QB --progress ${DB_FILE} \
    --no_teams --target_calc_stats "*" --hist_recent_games 3 \
    --player_team_stats passing_yds yds pts turnovers pens \
    --opp_team_stats def_int def_sacks op_passing_yds op_pts op_yds \
    --hist_recent_mode ma --current_extra venue is_home \
    -f nfl_QB.csv

# team defence and win
dumpdata.sc --seasons 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 \
    --stats "*" --progress ${DB_FILE} \
    --no_players --target_calc_stats "*" --target_stat pts win --hist_recent_games 3 \
    --opp_team_stats "*" \
    --hist_recent_mode ma --current_extra venue is_home \
    -f nfl_team.csv
```

In [ ]:
import pandas as pd

RANDOM_SEED = 1
TRAINING_TIME = 600
VALIDATION_SEASON = 2022
RECENT_GAMES = 3
TRAINING_SEASONS = [
    2009,
    2010,
    2011,
    2012,
    2013,
    2014,
    2015,
    2016,
    2017,
    2018,
    2019,
    2020,
    2021,
]


In [ ]:
from datetime import datetime

from fantasy_py import PlayerOrTeam
from train_test import load_data, train_test, create_fantasy_model

TARGET = ("stat", "pts")
MODEL_NAME = "NFL-team-pts"
DATA_FILENAME = "nfl_team.csv"
P_OR_T = PlayerOrTeam.TEAM
INCLUDE_POS = None
TARGET_POS = None

# TARGET = ("stat", "win")
# MODEL_NAME = "NFL-team-win"
# DATA_FILENAME = "nfl_team.csv"
# P_OR_T = PlayerOrTeam.TEAM
# INCLUDE_POS = None
# TARGET_POS = None

# TARGET = ("calc", "dk_score_def")
# MODEL_NAME = "NFL-DEF-DK"
# DATA_FILENAME = "nfl_team.csv"
# P_OR_T = PlayerOrTeam.TEAM
# INCLUDE_POS = None
# TARGET_POS = None

# TARGET = ("calc", "dk_score_off")
# MODEL_NAME = "NFL-QB-DK"
# DATA_FILENAME = "nfl_QB.csv"
# P_OR_T = PlayerOrTeam.PLAYER
# INCLUDE_POS = False
# TARGET_POS = ["QB"]

# TARGET = ("calc", "dk_score_off")
# MODEL_NAME = "NFL-RB-DK"
# DATA_FILENAME = "nfl_RB.csv"
# P_OR_T = PlayerOrTeam.PLAYER
# INCLUDE_POS = False
# TARGET_POS = ["RB"]

# TARGET = ("calc", "dk_score_off")
# MODEL_NAME = "NFL-WRTE-DK"
# DATA_FILENAME = "nfl_WRTE.csv"
# P_OR_T = PlayerOrTeam.PLAYER
# INCLUDE_POS = True
# TARGET_POS = ["WR", "TE"]

raw_df, tt_data, one_hot_stats = load_data(
    DATA_FILENAME,
    TARGET,
    VALIDATION_SEASON,
    include_position=INCLUDE_POS,
    seed=RANDOM_SEED,
)

assert list(one_hot_stats.keys()) == ["extra:venue"]


for automl_type in ["tpot"]:  # , "autosk"]:
    dt_trained = datetime.now()

    model_filepath, performance = train_test(
        automl_type, MODEL_NAME, TARGET, tt_data, RANDOM_SEED, TRAINING_TIME, dt_trained
    )
    performance["season"] = VALIDATION_SEASON
    model = create_fantasy_model(
        MODEL_NAME,
        model_filepath,
        dt_trained,
        tt_data[0],
        TARGET,
        TRAINING_TIME,
        P_OR_T,
        RECENT_GAMES,
        automl_type,
        performance,
        TRAINING_SEASONS,
        seed=RANDOM_SEED,
        target_pos=TARGET_POS,
        training_pos=TARGET_POS,
        raw_df=raw_df,
        one_hot_stats=one_hot_stats,
    )
    model_filepath = model.dump(".".join([MODEL_NAME, TARGET[1], automl_type, "model"]))
    print(f"Model file saved to '{model_filepath}'")
